[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
# 【工作紀錄】歷年水庫水情資料爬取



2020年台灣梅雨季降雨不豐，加上遭逢56年來罕見夏季無颱風登陸的窘境，各地水庫紛紛見底，旱象至今未見緩解跡象。這場旱災有多嚴重？哪邊的水庫最渴？為了解答這些問題，就必須先拿到各地水庫的水情資料。

### 資料搜集方式
用Python的selenium自動化從「[台灣地區主要水庫蓄水量報告表](https://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx)」下載歷年資料。

### 資料限制
資料中並無水庫的地理位置資料，若要結合地理特徵，需另結合[水庫的地理資料](https://gic.wra.gov.tw/Gis/Gic/DataIndex/Data/Main.aspx)。

### 參考資料
- [GitHub]infographicstw/[reservoir-history-crawler專案](https://github.com/infographicstw/reservoir-history-crawler)

- 《天下雜誌》「[即時水情地圖》下雨了，台灣水庫解渴了嗎？｜互動專題｜天下雜誌](https://web.cw.com.tw/drought-2021/index.html)
」

## 資料爬取步驟
### 探索資料

1. 抓出網頁中的表格：從[台灣地區主要水庫蓄水量報告表](https://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx)網頁原始碼可以得知，每一年的資料都是一個表格，可以先用pandas把網頁中的表格拿下來，並處理成方便整理的DataFrame。
2. 整理DataFrame的header：仔細觀察DataFrame後發現，DataFrame的column包含雙層的header，因此先去除不需要的欄位名稱，並整理成單層header。
3. 去除不需要的欄位：DataFrame的最後一列都是附註，而最後一欄是空值，因此直接排除。

In [1]:
# 載入所需函式庫
import requests
import pandas as pd

In [2]:
# 從網頁中拿下表格並轉成 DataFrame
res = requests.get("https://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx")
df = pd.read_html(res.text)[0]

# 整理 header
columns = [i[1] for i in list(df.columns)]
df.columns = columns

# 去除不需要的資料
df = df.iloc[:-1, :-1]
df

,水庫名稱,有效容量(萬立方公尺),統計時間,集水區降雨量(毫米),進水量(萬立方公尺),出水量(萬立方公尺),與昨日水位差(公尺),水情時間,水位(公尺),有效蓄水量(萬立方公尺),蓄水量百分比(%)
0,石門水庫,20266.10,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,39.90,74.81,-0.14,2021-05-06(23時),214.56,3884.68,19.17 %
1,翡翠水庫,33550.50,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,79.20,218.20,-0.19,2021-05-06(23時),157.77,23054.70,68.72 %
2,寶山第二水庫,3147.18,起:2021-05-06(0時) 迄:2021-05-07(0時),0.20,10.50,14.24,-0.11,2021-05-06(23時),116.83,178.97,5.69 %
3,永和山水庫,2998.94,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,5.00,9.40,--,2021-05-06(23時),53.22,146.65,4.89 %
4,明德水庫,1222.05,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,1.41,1.41,--,2021-05-06(7時),48.18,120.42,9.85 %
5,鯉魚潭水庫,11446.01,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,30.76,49.13,--,2021-05-06(23時),253.24,558.30,4.88 %
6,德基水庫,18642.00,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,41.70,29.72,--,2021-05-06(7時),1327.48,702.28,3.77 %
7,石岡壩,149.31,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,38.90,48.70,--,2021-05-06(23時),271.74,25.82,17.29 %
8,霧社水庫,3752.88,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,25.85,51.84,--,2021-05-06(7時),984.45,257.65,6.87 %
9,日月潭水庫,12790.99,起:2021-05-06(0時) 迄:2021-05-07(0時),0.00,74.91,89.25,--,2021-05-06(7時),735.86,4077.32,31.88 %


### 自動化爬取資料
- 從[台灣地區主要水庫蓄水量報告表](https://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx)網頁原始碼可以看到，只要改變select的option值（value），就可以拿到新的資料表，因此寫一個自動化來處理抓取資料的任務。
- 觀察網站機制後發現，這個網站的日期選項並未對應月份資料，因此跑迴圈的時候不用特別設計，先用暴力的方法拿到資料再去除空值就好。

In [ ]:
# 載入所需函式庫
# !pip install selenium
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
columns = []
data = []

url = "https://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx"

driver = webdriver.Firefox(executable_path="the_path_to_your_driver")
# driver = webdriver.Chrome(executable_path="the_path_to_your_driver")
driver.get(url)

# 先選擇「所有水庫」
element = driver.find_element_by_xpath('/html/body/form/div[3]/div[1]/select[1]/option[2]').click()

for y in range(2010,2022):  # 改變年份的值
    select = Select(driver.find_element_by_id('ctl00_cphMain_ucDate_cboYear'))
    select.select_by_value('{}'.format(y))
    
    for m in range(1,13):  # 改變月份的值
        select = Select(driver.find_element_by_id('ctl00_cphMain_ucDate_cboMonth'))
        select.select_by_value('{}'.format(m))
        
        for d in range(1,32):  # 改變日期的值
            try:
                select = Select(driver.find_element_by_id('ctl00_cphMain_ucDate_cboDay'))
                select.select_by_value('{}'.format(d))
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,
                                             '//*[@id="ctl00_cphMain_btnQuery"]'))).click()
                
                res = driver.page_source
                df = pd.read_html(res)[0]
                columns = [i[1] for i in list(df.columns)]
                df.columns = columns
                data.append(df.iloc[:-1, :-1])
                sleep(0.75)
            
            except StaleElementReferenceException:
                pass

# 合併所有DataFrame並存成csv檔
df_data = pd.concat(data)
df_data.columns = columns[:-1]
df_data.to_csv("reservoir_history_data.csv", index=False)
